# Exercise set 1: Deadline: 21-4-2024 23:59.
Hand in this notebook with output. Make sure that it is able to run and produce all the figures and results you show. Also, use the text boxes to answer the questions and interpret your results, relating them to the course materials.


Exercises made by Oliver Gurney-Champion. Please contact us via Canvas, or e-mail directly to:
Oliver: o.j.gurney-champion@amsterdamumc.nl
Matthan: m.w.a.caan@amsterdamumc.nl
Daisy: daisyvandenberg@amsterdamumc.nl
Daan: d.kuppens@amsterdamumc.nl

These are a large set of challenging exercises, for which you will get 3 weeks to complete. I would strongly advise you to stick to the suggested schedule, which will ensure you have sufficient knowledge to answer the questions when completing them, and finalize all questions in time.

Note that the networks will be lite and can run on your local computer/laptop in short time (minutes). There is no need as yet to run this on Surf, although we highly encourage you to make sure Surf works for you (for exercise sets 2 and 3).

# 1: Program network in PyTorch. (60%)
During the class, a brief introduction was given to quantitative imaging. In this exercise, you will program your first neural network that will help estimate quantitative MRI parameters from quantitative data. In particular, we will be looking at the intra-voxel incoherent motion (IVIM) model for diffusion-weighted MRI:

S(b)=S_0×( (1-f)×e^-b×D^ +f×e^-b×D*^ )                                                                    [1]

With S the measured signal, S0 the baseline signal at S(b=0), f the perfusion fraction, D the diffusion coefficient and D* the pseudo diffusion coefficient. For more information on what the model means exactly and how it is used clinically, I would suggest reading "Introduction to IVIM MRI | Radiology Key" (https://radiologykey.com/introduction-to-ivim-mri/). But for the purpose of this exercise, it is just a model.

Normally, f, D and D* (named Dp in the code) are obtained by fitting S(b) using least-squares fitting. But these approaches are known to be prone to noise in the data and often produce poor estimates.

Therefore, you will write a neural network that predicts f from a given S(b). There are great tools available that take care of training models, such as PyTorch, Karas and Tensorflow. However, for the purpose of this exercise, you will make use of PyTorch. Moreover, we will use weights-and-biasses to keep track of how training is going.

# Wednesday 3-4-2024
At https://github.com/oliverchampion/AI_for_medical_imaging_course you will find the Python assignment. To help visualize progress and to isolate certain snippets of code, we wrote this as a Jupyter Notebook (exercise1.ipynb). As you can see, we have already provided a data-generator, some plotting tools to plot the training progress. The notebook should run as is and train a neural network! For your first lecture, we suggest you (Wednesday)
-	Install all prerequisites in your virtual enviroment (requierements.txt)
-	Start a WandB account @ https://wandb.ai/  you will need to log in when running the script
-	Go through the script to see whether you understand what happens.
-	Train your first neural network .
-	Visualize the results on your WandB page.

## Import requiered packages
imports the packages and sets the random seed

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import random
import numpy as np
import helper_functions as hf
import torch
import os
import torch.nn as nn
from matplotlib import pyplot as plt

# set random seed
seed =42

login to your free wandb account. Note you will need to set up your account on https://wandb.ai/authorize
wandb allows you to keep track of your neural network training.

In [ ]:
import wandb
os.environ["WANDB_SILENT"] = "true"
%env WANDB_SILENT=True
wandb.login()

## Simulate and view the IVIM data
This allows you to study what the data looks like in jupyter.

In [ ]:
# set b-values at which we "measure" (i.e. simulate signal)
bvalues=[0, 10, 20, 30, 50, 75, 100, 150, 300, 500, 700, 1000]

## Set the random seeds for reproducibility
np.random.seed(seed)
random.seed(seed)

## Loading the (simulated) dataset
data_sim, D, f, Dp = hf.sim_signal(SNR=(10,30),bvalues=bvalues,sims=30,seed=np.random.randint(1,10000))

## plotting some curves and data for visualisation
fig, axs = plt.subplots(2, 2)
for i in range(2):
    for j in range(2):
        axs[i, j].plot(bvalues, data_sim[i+10*j,:], 'o')
        datapred=hf.ivim(np.arange(0,np.max(bvalues)), D[i+10*j], f[i+10*j], Dp[i+10*j], 1)
        axs[i, j].plot(np.arange(0,np.max(bvalues)), datapred)
        axs[i, j].set_ylim(0, 1.2)
        axs[i, j].set(xlabel='b-value (s/mm2)', ylabel='normalised signal')
plt.legend(('noisy data', 'true curve'))

## Splitting the data into training and validation
Here, we split our data into a training set, validation set and test set. Note that the current implementation only uses the training set and it is up to you (in your exercises) to also implement the validation and test run. At this point, we already split the data for you.

In [ ]:
#determine
batch_size = 16
with wandb.init(project="AI_for_medical_imaging", job_type="visualize data") as run:
    data_sim = hf.sim_signal(SNR=(10,40),bvalues=bvalues,sims=10000,seed=np.random.randint(1,10000))
    # Only for visualisation purposes: here we create our "Artifact" in wandb --> this allows viewing the data in your wandb account
    for i in range(4):
        #make b-value data pairs
        example_data=[[x,y] for (x,y) in zip(bvalues,data_sim[0][i])]
        # put it in a table
        table = wandb.Table(data=example_data, columns=["b-values", "signal"])
        #tell wandb to plot the table
        wandb.log({"data_plot " + str(i): wandb.plot.scatter(table, "b-values", "signal")})

    # here we split the data into train (70%), test (15%) and validation (15%) sets
    #split = int(np.floor(len(data_sim[0]) * 0.7))
    train_set, test_set, val_set = torch.utils.data.random_split([[data_sim[0][i,:],data_sim[1][i],data_sim[2][i],data_sim[3][i]] for i in range(len(data_sim[3]))],[0.7,0.15,0.15])
    #split = int(np.floor(len(rest) * 0.5))
    #test_set, val_set = torch.utils.data.random_split([[rest[0][i,:],rest[1][i],rest[2][i],rest[3][i]] for i in range(len(rest[3]))],[split, len(rest[0]) - split])

    # train loader loads the trianing data. We want to shuffle to make sure data order is modified each epoch and different data is selected each epoch.
    trainloader = torch.utils.data.DataLoader(train_set,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   drop_last=True)
    # validation data is loaded here. By not shuffling, we make sure the same data is loaded for validation every time. We can use substantially more data per batch as we are not training.
    inferloader = torch.utils.data.DataLoader(val_set,
                                   batch_size=128,
                                   shuffle=False,
                                   drop_last=True)
        # validation data is loaded here. By not shuffling, we make sure the same data is loaded for validation every time. We can use substantially more data per batch as we are not training.
    testloader = torch.utils.data.DataLoader(test_set,
                                   batch_size=128,
                                   shuffle=False,
                                   drop_last=True)

## Design a neural network

In [ ]:
# funtion for stacking the layers and making the model
def make_model(n_inputs=5,n_hidden=1,n_outputs=1):
    #initialize the model object
    model = nn.Sequential()
    # fill out the model with hidden layers.
    for i in range(n_hidden):
        # as we loop, we add hidden layers
        model.add_module('layer_linear'+str(i), nn.Linear(n_inputs, n_inputs))
        # we also add a ReLu layer
        model.add_module('layer_ReLu'+str(i), nn.ReLU())
    #and a final output layer
    model.add_module('last_layer',nn.Linear(n_inputs, n_outputs))
    # to ensure positive predictions, we end with a ReLu function before giving output
    model.add_module('last',nn.Sigmoid())
    model.apply(init_weights)
    return model

# function for initializing network weights for individual layers
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)

train your first network

In [ ]:
def train_network(name, epochs=20, learningrate=0.01, hidden_layers=2, seed=42, optimizer='SGD'):


    model = make_model(n_inputs=len(bvalues), n_hidden=hidden_layers, n_outputs=1)

    # initialize model --> we did this above, but during the exercise, you might be re-running this part of the script several times with different settings. This way we make sure you re-initiate the training and don't continue in the last model
    model.apply(init_weights)

    # initialize wandb
    wandb.init(
            project="AI_for_medical_imaging", job_type="training", name=name)

    # set random seed for reproducibility
    torch.manual_seed(seed)

    # probe available devices
    if torch.cuda.is_available():  # GPU operation have separate seed
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.determinstic = True
        torch.backends.cudnn.benchmark = False

    # Set default device. If GPU is available, the network will be trained on the GPU. Note that further down in the code, stuff will be sent ".to(device)" to make sure it is available on the GPU.
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # define the loss of the network (mean square error)
    loss_module = nn.MSELoss(reduction='mean').to(device)

    # the optimizer determines how strongly to update the network's weights based on the calculated loss.
    if optimizer == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=learningrate)
    else:
        raise NotImplementedError('this optimizer is not implemented yet...')

    # loop over epochs
    for epoch in range(epochs):
        # initiate losses to 0
        train_loss_f = 0
        val_loss_f = 0
        # set model to training such that forward passes are remembered (requiered for backpropogating the loss)
        model.train()
        #loop over all training data0
        SD_train = 0
        sys_train = 0
        for x in trainloader:
            # reset the gradients to zero
            optimizer.zero_grad()
            # get data (x[0]) and put the data on the GPU if available
            batch=x[0].to(device)
            # get the reference f (x[2]) --> note x[1] and x[3] are D and Dp respectively
            f_ref = torch.flatten(x[2].to(device))
            # put the data through the neural network
            f_pred = torch.flatten(model.forward(batch))
            # calculate loss (compare predicted f to the ground trueth)
            loss_f = loss_module(f_pred, f_ref)
            #add found loss to the train loss, to keep track of the loss this epoch
            train_loss_f += loss_f.item()
            # propogate the loss through the network (calculate d_wights/d_loss)
            loss_f.backward()
            # update all weights accoording to their derrivatives to the loss.
            optimizer.step()
            # calculate the standard deviation and systematic error on the trianing data
            SD, sys = hf.error_metrics(f_pred.cpu().detach().numpy(),f_ref.cpu().detach().numpy())
            # add the errors to ultimately calculate their mean over the trianing data. calculating mean SDs goes via the Root Mean Squares. So add SDs squared
            SD_train += SD**2
            sys_train += sys
        # now divide by the total amount of training data to calculate the mean (sys error) and square of mean (SD).
        SD_train = np.sqrt(SD_train/inferloader.__len__())
        sys_train = sys_train/inferloader.__len__()
        # after training, set model to evaluation mode
        model.eval()
        # initialize error_metrics
        SD_val=0
        sys_val=0
        ######################your code here for validation loss#########################
        #make b-value data pairs: Note these currently contain the f_ref and f_pred from the trianing data. You may want to swap to validation data once implemented
        example_data=[[x,y] for (x,y) in zip(f_ref.cpu().detach().numpy(),f_pred.cpu().detach().numpy())]
        # put it in a table
        table = wandb.Table(data=example_data, columns=["f_ref", "f_pred"])
        #tell wandb to plot the table
        # note that some parameters are being logged which you still need to define in the validation loop!
        wandb.log({"loss/train": train_loss_f/trainloader.__len__(),"loss/val": val_loss_f/inferloader.__len__(),"error/random error":SD_train,"error/systematic error":sys_train,"data_plot epoch " + str(epoch): wandb.plot.scatter(table, "f_ref", "f_pred", title=f'epoch{epoch}')})

        ## print output in terminal. Only useful for debugging when WandB does not work
        #print('epoch = ' + str(epoch) + ' train loss =' + str(train_loss_f/trainloader.__len__()) +' val loss =' + str(val_loss_f/inferloader.__len__()) + 'the systematic error is ' + str(sys_val) + ' and the random error is ' + str(SD_val))

train_network('test')

# Friday 5-4-2024
For these exercises, you will want to produce new cells that generate the outputs. The notebook cells should run and produce the figures without input of the examiners.

## 1A-C: train the neural network
Adapt the script above for your exercises A-C such that it produces the desired results and plots. Describe the results/intepertation in this text baloon.

A.	The current network implementation only looks at training data. This means that the network’s performance is over-estimated. Please use the validation set to monitor performance during training (note that we have already put the model on evaluation mode in line 59). At what point is the network fully trained? Explain how you know this. Show the effect from overfitting and underfitting.
B.	Similarly, use the test dataset to test for final performance. Explain why this is needed.
C.	Currently, standard gradient descent optimizer is being used to train the network, with a learning rate of 0.01. Investigate the performance of the network for different  optimizer (i.e. adam loss was discussed in the lecture) and explain what you see. What does the Adam  optimizer do differently from the SGD  optimizer that would make it perform better/differently?


D.	Return back to SGD. Test how does the performance depends on the learning rate. Plot the performance. This can be done by plotting the systematic (sys_val) and random (SD_val) errors as function of learning rate (10<LR<0.0000001; steps in order of magnitude; e.g. 10, 1, 0.1, ….). Explain what you see.

E.	Plot the performance (sys_val and SD_val) as function of the width (5, 10, 20, 50, 100) and depth (1, 2, ..., 8) of the network. What is the effect of width and depth on the network performance? How wide/deep would you advice us to make the network?

F.	Train the network for 1000 epochs. At what point in the network fully trained? How do you see this?

G.Try different batch sizes (1, 4, 16, 64, 128, 516, 1024).Explain the behavior of the network you see.What is the effect of having smaller batches? And larger batches?

# Wednesday 10-4-2024
H.	Currently, the model uses a Relu activation function. Test the effect of different activation funtions on the network performance. Show how well does a sigmoid or ELU work (i.e. what is the effect on performance)?

I.	Let the network also predict D and Dp (note x[1] and x[3] are D and Dp respectively). Show the loss curve of D and Dp and explain how you can see that they have been implemented properly/the model is learning them.
o	The network will need more than 1 output --> tip, use loss_D.backward(retain_graph=True) to propogate all 3 losses backward in turn
o	Alternatively, you could train 3 networks simultaniously

J.	Use the sigmoid in the final layer to constrain 0.5e-3 <D< 3.5e-3; 0<f<1; 5e-3<D*<100e-3. Explain how you did this.

# Bonus
So far, you have been working with simulated data (taken care of by Data_loader.py). For such data, we know the ground truth values. However, in vivo, we have no way of knowing the ground truth. How will our network perform?

K.	Use the network, as trained in (4) and apply it to real-world data which is provided by running “test_in_vivo.py”. Note that alongside your plot (the first), also a conventional least squares fit is provided as a reference. Show the resulting parameter maps. How does your approach compare? Why do you think your particular approach would look better/worse?

L.	Ideally, you would train your network on real-world data. However, in this particular case, it is hard to get gold standard references. Luckily, we can use our understanding of physics, and of how stuff “should behave” to work our way around this. You will redesign your network loss, such that it can train on data without any gold standard references! Instead of placing the L2 loss on f_pred v.s. f_ref. Currently, the network is learning to minimize the difference between predicted fpred and the ground truth referene ftrue. In vivo, we may not have these references. To overcome this, we will now introduce a physics-informed loss. Use the IVIM equation [1] to propogate the predictions (D, f and Dp) into the signal space (S). Then, take e.g. the mean-square-error between the predicted signal and the input signal. Note that you will need to use torch functions (instead of numpy functions) to ensure you can backpropogate the loss through the equation into the network.

M.	Optimize the network’s training using the real data provided (“test_in_vivo.py”; datatrain). Evaluate the network on the same data as in 1 (data, valid_id, bvalues = dl.load_real_data(eval=True)). How does it perform?


In [ ]:
datatrain, bvalues = hf.load_real_data(eval=False)
dataval, valid_id, bvalues = hf.load_real_data(eval=True)
print('training data is ' + str(len(datatrain)) + ' long')

''''your code here'''
### use datatrain to train your network.
## then test your network using dataval.
''''if you manage to make predictions of D, f and Dp, the following code will allow you to plot them:'''

hf.plot_example(np.squeeze(D_out), valid_id,0.003)
hf.plot_example(np.squeeze(f_out), valid_id,0.7)
hf.plot_example(np.squeeze(Dp_out), valid_id,0.1)
### you can compare supervised and self-supervised fits